In [44]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv
import time


In [2]:
df = pd.read_csv('data/reddit_questions.csv')

In [3]:
df.head()

,Unnamed: 0,Subreddit,Questions,Human_Answer
0,0,TrueAskReddit,Why does the president seem so personally inte...,"I've been reading through these comments, and ..."
1,1,TrueAskReddit,How did you end up in your career path? Are yo...,i'm a massage therapist. not the seaweed wrap ...
2,2,TrueAskReddit,Why does the world come together spending hund...,Being blunt but here are my two big factors:\n...
3,3,TrueAskReddit,What atrocities are occurring around the world...,My heart is particularly called out to the Con...
4,4,TrueAskReddit,How can a male boss better lookout and protect...,By firing the guy you already did more than yo...


The following is only necessary when working with the first version of 'reddit_questions.csv'

In [4]:
mask = df['Human_Answer'] == '[deleted]'

d = df[~mask]

In [5]:
df.shape

(4615, 4)

In [6]:
d.shape

(4400, 4)

In [7]:
load_dotenv()  

True

In [8]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [10]:
test_prompt = "Hi,how are you?"

In [11]:
response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role':'user','content':test_prompt}],
    temperature = 0.5,
    max_tokens = 500

)

In [12]:
response['choices'][0]['message']['content']

"Hello! I'm an AI, so I don't have feelings, but I'm here to help. How can I assist you today?"

In [15]:
davinci_prompts = list(d['Questions'][:10])

In [31]:
dv_response = openai.Completion.create(
    model = 'text-curie-001',
    prompt = davinci_prompts,
    temperature = 0.6,
    max_tokens = 250
)

In [30]:
dv_response['usage']

<OpenAIObject at 0x29b2a310110> JSON: {
  "prompt_tokens": 289,
  "completion_tokens": 1321,
  "total_tokens": 1610
}

In [20]:
dv_response['choices'][0]

<OpenAIObject at 0x29b2a0a85f0> JSON: {
  "text": "\n\nThe president likely feels a personal responsibility to respond to the use of chemical weapons in Syria. This is due to the moral imperative of preventing the use of weapons of mass destruction, as well as the need to uphold international norms of behavior. The president may also feel a sense of personal responsibility to protect innocent civilians in Syria who have been affected by the conflict.",
  "index": 0,
  "logprobs": null,
  "finish_reason": "stop"
}

In [32]:
for i in range(len(davinci_prompts)):
    print(davinci_prompts[i] + ':\n', dv_response['choices'][i]['text'],'\n')

Why does the president seem so personally interested in action against Syria?:
 

Some have speculated that the president is personally interested in action against Syria because he is a strong supporter of the anti-Assad rebels. Others have suggested that the president is concerned about the possible humanitarian consequences of a military intervention in Syria. 

How did you end up in your career path? Are you happy with it?:
 

I initially pursued a degree in biomedical engineering, but I quickly realized that I wanted to work in the health care industry. After completing my undergraduate degree, I worked as a research associate in a biomedical research lab. I then moved to a hospital where I worked as a research scientist. After about five years, I decided that I wanted to move into a healthcare leadership role. I pursued a master's degree in healthcare leadership and management, and I now work as a director of research and development in a healthcare company. I am very happy with 

In [26]:
response_list = [res['text'].strip() for res in dv_response['choices']]

In [27]:
response_list

['The president likely feels a personal responsibility to respond to the use of chemical weapons in Syria. This is due to the moral imperative of preventing the use of weapons of mass destruction, as well as the need to uphold international norms of behavior. The president may also feel a sense of personal responsibility to protect innocent civilians in Syria who have been affected by the conflict.',
 'I ended up in my career path because I have always been interested in business and finance. After college, I took a job in banking and eventually moved into a finance-related role. I am very happy with my career path because I love the work I do and the challenges it presents. I also enjoy the people I work with and the opportunity to learn and grow.',
 'The answer to this question is complex and has many factors. One of the primary motivations for the world to come together and spend large amounts of money for a rescue effort is the potential for a large loss of life if the effort is no

In [33]:
d.head()

,Unnamed: 0,Subreddit,Questions,Human_Answer
0,0,TrueAskReddit,Why does the president seem so personally inte...,"I've been reading through these comments, and ..."
1,1,TrueAskReddit,How did you end up in your career path? Are yo...,i'm a massage therapist. not the seaweed wrap ...
2,2,TrueAskReddit,Why does the world come together spending hund...,Being blunt but here are my two big factors:\n...
3,3,TrueAskReddit,What atrocities are occurring around the world...,My heart is particularly called out to the Con...
4,4,TrueAskReddit,How can a male boss better lookout and protect...,By firing the guy you already did more than yo...


In [34]:
d['Marie_answer'] = 0

C:\Users\kalya\AppData\Local\Temp\ipykernel_8136\2787529008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['Marie_answer'] = 0


In [35]:
d.head()

,Unnamed: 0,Subreddit,Questions,Human_Answer,Marie_answer
0,0,TrueAskReddit,Why does the president seem so personally inte...,"I've been reading through these comments, and ...",0
1,1,TrueAskReddit,How did you end up in your career path? Are yo...,i'm a massage therapist. not the seaweed wrap ...,0
2,2,TrueAskReddit,Why does the world come together spending hund...,Being blunt but here are my two big factors:\n...,0
3,3,TrueAskReddit,What atrocities are occurring around the world...,My heart is particularly called out to the Con...,0
4,4,TrueAskReddit,How can a male boss better lookout and protect...,By firing the guy you already did more than yo...,0


In [49]:
questions = [question for question in d.Questions]
batch_size = 20
question_batches = [questions[i:i + batch_size] for i in range(0, len(questions), batch_size)]

response_list = []
count = 0
for batch in question_batches:
    
    maries_response = openai.Completion.create(
        model = 'text-curie-001',
        prompt = batch,
        temperature = 0.6,
        max_tokens = 250
    )
   
    
    batch_response = [res['text'].strip() for res in maries_response['choices']]
    
    
    
    response_list += batch_response
    count+=1

    #sleep time
    time.sleep(1)
    print(f'finished batch number{count}')


finished batch number1
finished batch number2
finished batch number3
finished batch number4
finished batch number5
finished batch number6
finished batch number7
finished batch number8
finished batch number9
finished batch number10
finished batch number11
finished batch number12
finished batch number13
finished batch number14
finished batch number15
finished batch number16
finished batch number17
finished batch number18
finished batch number19
finished batch number20
finished batch number21
finished batch number22
finished batch number23
finished batch number24
finished batch number25
finished batch number26
finished batch number27
finished batch number28
finished batch number29
finished batch number30
finished batch number31
finished batch number32
finished batch number33
finished batch number34
finished batch number35
finished batch number36
finished batch number37
finished batch number38
finished batch number39
finished batch number40
finished batch number41
finished batch number42
f

In [50]:
len(response_list)

4400

In [51]:
d['Marie_answer'] = response_list

C:\Users\kalya\AppData\Local\Temp\ipykernel_8136\189437120.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['Marie_answer'] = response_list


In [55]:
d[1045:1049]

,Unnamed: 0,Subreddit,Questions,Human_Answer,Marie_answer
1096,1096,AskScienceFiction,[Indiana Jones] When Indy saw that the Nazis w...,There are 2 main reasons\n\n1) It wasn't a Naz...,Indy's goal was always to find the Ark and sto...
1097,1097,AskScienceFiction,[Marvel Comics] Why did Norse paganism fade aw...,"Canonically, he left the earth in shame over t...",There is no one answer to this question. Some ...
1098,1098,AskScienceFiction,[Star Wars] Why do people in the Star Wars uni...,"Less than 10,000 jedi at their height of power...",The Jedi were disbanded after the death of the...
1099,1099,AskScienceFiction,[Star Wars] HK-47 mentions focusing on strong ...,"Trained Jedi can read your surface thoughts, s...","No, that does not mean attacking while unbelie..."


In [59]:
d.head()

,Unnamed: 0,Subreddit,Questions,Human_Answer,Marie_answer,human_answer
0,0,TrueAskReddit,Why does the president seem so personally inte...,"I've been reading through these comments, and ...",The president has been very interested in acti...,"I've been reading through these comments, and ..."
1,1,TrueAskReddit,How did you end up in your career path? Are yo...,i'm a massage therapist. not the seaweed wrap ...,I majored in Political Science and minored in ...,i'm a massage therapist. not the seaweed wrap ...
2,2,TrueAskReddit,Why does the world come together spending hund...,Being blunt but here are my two big factors:\n...,The answer is that we have a sense of responsi...,Being blunt but here are my two big factors:\n...
3,3,TrueAskReddit,What atrocities are occurring around the world...,My heart is particularly called out to the Con...,There are many atrocities occurring around the...,My heart is particularly called out to the Con...
4,4,TrueAskReddit,How can a male boss better lookout and protect...,By firing the guy you already did more than yo...,The best way for a male boss to better lookout...,By firing the guy you already did more than yo...


In [57]:
d.to_csv('data/gpt_answers_v1.csv')